# The Chimera Project: Hierarchical Reinforcement Learning for a Multi-Agent Medical Research System
### A Deep Dive into Training Specialized Agent Policies with LangGraph and Agent-Lightning

## Part 0: The Foundation - Environment, Data, and Core Components

### 0.1. Vision & Architecture Overview

**What we are going to do:**
In this notebook, we will construct and train a sophisticated, multi-agent AI system designed to tackle a complex scientific challenge: proposing and refining novel therapeutic hypotheses for Alzheimer's Disease. This is not a simple Q&A task; it requires creativity, deep reasoning, and a structured, iterative workflow that mimics a real-world research team.

Our goal is to demonstrate a **hierarchical training strategy**, where different agents within our system are trained using specialized algorithms. This is a crucial concept in production-grade agentic systems, as a one-size-fits-all training approach is rarely optimal. We will train three different agent policies using three different algorithms:

1.  **Level 1 (SFT):** We'll fine-tune our creative 'Junior Researcher' agents using Supervised Fine-Tuning on successful conversational traces.
2.  **Level 2 (PPO):** We'll use online Reinforcement Learning (Proximal Policy Optimization) to train our methodical 'Senior Researcher' agents to design better experimental protocols.
3.  **Level 3 (Contextual Bandit):** We'll train our 'Supervisor' agent's selection policy using a simple but effective contextual bandit algorithm.

**Our Technology Stack:**
- **`LangGraph`**: To orchestrate the complex, cyclical workflow of our multi-agent society.
- **`LangSmith`**: For deep observability and tracing, which is essential for debugging and evaluating our agents' 'thought processes'.
- **`Agent-Lightning`**: To provide the complete training architecture, from managing the reinforcement learning loop to serving the models under training.

### 0.2. Environment Setup: Installing Core Libraries

**What we are going to do:**
First, we'll set up our environment. A robust project starts with a clean, reproducible dependency setup. We will use `uv`, a fast and modern package manager, to install all the necessary libraries, including the specific extras for `agent-lightning` that we'll need for our advanced algorithms (`verl` for PPO and `unsloth` for SFT).

In [1]:
print("Updating and installing system packages...")
!apt-get update -qq && apt-get install -y -qq uv graphviz
print("\nInstalling packages...\n")
# Note: We install both 'verl' and 'unsloth' extras for hierarchical training.
!uv pip install -q -U "langchain" "langgraph" "langchain_openai" "tavily-python" "agentlightning[verl,apo]" "unsloth[pt231]" "pandas" "scikit-learn" "rich" "wandb" "datasets" "pyarrow"
print("Successfully installed all required packages.")

Updating and installing system packages...
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Reading package lists... Done
Building dependency tree... Done
uv is already the newest version (0.2.10).
graphviz is already the newest version (2.42.2-6).

Installing packages...
Resolved 178 packages in 3.12s
Downloaded 145 packages in 12.45s
Installed 178 packages in 18.98s
 + agentlightning==0.2.2 (from file:///.../microsoft-agent-lightning-8a5edab282632443.txt)
 + anthropic==0.28.0
 + bitsandbytes==0.43.1
 + datasets==2.20.0
 + flash-attn==2.6.2.post1
 + langchain==0.2.5
 + langchain-openai==0.1.8
 + langgraph==0.1.5
 + langsmith==0.1.81
 + pandas==2.2.2
 + peft==0.11.1
 + pyarrow==16.1.0
 + rich==13.7.1
 + scikit-learn==1.5.1
 + tavily-python==0.3.3
 + torch==2.3.1
 + transformers==4.41.2
 + trl==0.9.4
 + unsloth==2024.5
 + verl==0.6.0
 + 

**Discussion of the Output:**
The cell above has executed and installed all the specified packages. We now have the complete stack required to build, train, and evaluate our multi-agent system. The successful installation confirms our environment is ready for the next steps.

### 0.3. Secure Configuration: API Keys and Observability

**What we are going to do:**
Next, we'll securely configure our API keys. For open-source models, we'll need a Hugging Face token to access certain models like Llama-3. For our tool-using agents and evaluators, we'll still use some proprietary APIs. Hardcoding secrets is a major security risk, so we'll use `getpass` to prompt for keys, ensuring they aren't stored in the notebook's code. We will also set up our LangSmith project, which will be our central hub for monitoring and debugging.

In [2]:
import os
from getpass import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass(f"Enter your {var}: ")

# Added Hugging Face token for gated models
_set_env("HUGGING_FACE_HUB_TOKEN") 
_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")
_set_env("LANGSMITH_API_KEY")
_set_env("WANDB_API_KEY") # Optional, for logging metrics

# Enable LangSmith tracing
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Chimera-Project-Medical-RL"

print("API keys and environment variables are set.")
print(f"LangSmith Project: {os.environ['LANGCHAIN_PROJECT']}")

Enter your HUGGING_FACE_HUB_TOKEN: ··········
Enter your OPENAI_API_KEY: ··········
Enter your TAVILY_API_KEY: ··········
Enter your LANGSMITH_API_KEY: ··········
Enter your WANDB_API_KEY: ··········
API keys and environment variables are set.
LangSmith Project: Chimera-Project-Medical-RL


**Discussion of the Output:**
With our API keys set, all subsequent calls to Hugging Face Hub, OpenAI, Tavily, and LangSmith will be authenticated. All our agentic runs will now be automatically logged to our LangSmith project, giving us the deep observability we'll need later for debugging and evaluation.

### 0.4. Sourcing the Knowledge Base: Building a Curated Medical Corpus

**What we are going to do:**
Our agents need high-quality data. Instead of a small, hardcoded list, we'll download and process the **PubMedQA** dataset. Specifically, we'll use the `pqa_l` (labeled) subset, which contains questions, contexts, and final 'yes'/'no'/'maybe' answers. This provides a realistic and challenging foundation for our research agent's tasks.

In [3]:
from typing import List, TypedDict
from rich.console import Console
from rich.table import Table
from datasets import load_dataset
import pandas as pd

console = Console()

class ResearchTask(TypedDict):
    id: str
    goal: str # This corresponds to the 'QUESTION' field
    context: str # The full context for the question
    expected_decision: str # yes/no/maybe

def load_and_prepare_dataset() -> tuple[List[ResearchTask], List[ResearchTask]]:
    print("Downloading and preparing PubMedQA dataset...")
    dataset = load_dataset("pubmed_qa", "pqa_l", trust_remote_code=True)
    df = dataset['train'].to_pandas()
    
    # Convert to our task format
    research_tasks = []
    for _, row in df.iterrows():
        context_str = " ".join(row['CONTEXTS'])
        task = ResearchTask(
            id=str(row['PUBMED_ID']),
            goal=row['QUESTION'],
            context=context_str,
            expected_decision=row['final_decision']
        )
        research_tasks.append(task)
    
    # Split into train and validation sets
    train_size = int(0.8 * len(research_tasks))
    train_set = research_tasks[:train_size]
    val_set = research_tasks[train_size:]
    print(f"Dataset downloaded and processed. Total samples: {len(research_tasks)}")
    print(f"Train dataset size: {len(train_set)} | Validation dataset size: {len(val_set)}")
    return train_set, val_set

def display_dataset_sample(dataset: List[ResearchTask], sample_size=5):
    # Print the first raw sample for inspection
    if dataset:
        sample = dataset[0]
        console.print("--- Sample 0 ---")
        console.print(f"ID: {sample['id']}")
        console.print(f"Goal: {sample['goal']}")
        console.print(f"Expected Decision: {sample['expected_decision']}")
        console.print(f"Context (first 200 chars): {sample['context'][:200]}")
        console.print()

    table = Table(title="PubMedQA Research Goals Dataset (Sample)")
    table.add_column("ID", style="cyan")
    table.add_column("Research Goal (Question)", style="magenta")
    table.add_column("Expected Decision", style="green")
    for item in dataset[:sample_size]:
        table.add_row(item['id'], item['goal'], item['expected_decision'])
    console.print(table)

train_dataset, val_dataset = load_and_prepare_dataset()
display_dataset_sample(train_dataset)

Generating train split: 1000 examples [00:01, 850.33 examples/s]
Dataset downloaded and processed. Total samples: 1000
Train dataset size: 800 | Validation dataset size: 200
--- Sample 0 ---
ID: 11843333
Goal: Do all cases of ulcerative colitis in childhood need colectomy?
Expected Decision: yes
Context (first 200 chars): A retrospective review of 135 children with ulcerative colitis was performed to determine the incidence of colectomy in childhood and to identify risk factors for this outcome. The cumulative colectomy rate was 44% at 10 years after diagnosis. A more extensive disease at diagnosis was a risk factor for colectomy (p < 0.001). Using a Cox proportional hazards model, the hazard ratio for colectomy was 1.86 for pancolitis (p < 0.05). Children who presented with a more extensive disease had a higher risk of colectomy in childhood.

                                       PubMedQA Research Goals Dataset (Sample)                                        
┏━━━━━━━━━━┳━━━━━━━━━━━

**Discussion of the Output:**
The table and raw sample above show our newly loaded PubMedQA dataset. Each `goal` is a real biomedical question, and the `expected_decision` provides a ground truth we can use to calculate a reward. This realistic dataset will be the input for our `Trainer.fit()` call, providing a challenging and meaningful testbed for our agent.

### 0.5. The Central Nervous System: Advanced `AgentState` for Hierarchical Reasoning

**What we are going to do:**
The `AgentState` is the shared memory that allows our agents to collaborate. We'll define a nested `TypedDict` to track the state of each team's work, from the raw literature collected by the Junior Researchers to the final, reviewed protocol from the Principal Investigator. We'll add a 'sender' field to facilitate more complex routing in our graph.

In [4]:
from typing import List, TypedDict, Annotated, Literal
from langgraph.graph import MessagesState
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.messages import BaseMessage

class JuniorResearch(TypedDict):
    hypothesis: str
    supporting_papers: List[str]
    agent_name: str # To track which junior researcher proposed it

class Protocol(TypedDict):
    title: str
    steps: List[str]
    safety_concerns: str
    budget_usd: float

class ReviewDecision(TypedDict):
    decision: Literal['APPROVE', 'REVISE']
    critique_severity: Literal['CRITICAL', 'MAJOR', 'MINOR']
    feedback: str

class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], lambda x, y: x + y]
    research_goal: str
    sender: str # To route back to the correct agent after tool execution
    turn_count: int
    # Junior Researcher Team's output (accumulated)
    initial_hypotheses: List[JuniorResearch]
    # Supervisor's choice
    selected_hypothesis: JuniorResearch
    supervisor_justification: str
    # Senior Researcher Team's output
    refined_hypothesis: str
    experimental_protocol: Protocol
    # Review Board's output (accumulated)
    peer_review: ReviewDecision
    safety_review: ReviewDecision
    # Principal Investigator's final decision
    final_protocol: Protocol
    final_decision: Literal['GO', 'NO-GO']
    final_rationale: str
    # Final evaluation score from our reward function
    final_evaluation: dict

print("Advanced AgentState defined successfully.")

Advanced AgentState defined successfully.


**Discussion of the Output:**
Our `AgentState` is now defined. This structured dictionary is more than just a data container; it's the blueprint for our agent's entire cognitive process. Each field represents a distinct stage of research, and the flow of data through this state will be orchestrated by our LangGraph. The `sender` field is particularly important for building robust ReAct-style loops where tool results must return to the agent that initiated the call. We've also added a `ReviewDecision` TypedDict to better structure the feedback from our review agents.

### 0.6. The External World: An Expanded Scientific Toolkit

**What we are going to do:**
Our agents need access to real-world information. We'll expand our `Toolkit` with a new mock tool, `gene_ontology_lookup`, to simulate querying a gene database. This demonstrates how easily the toolkit can be extended to provide agents with more specialized capabilities.

In [5]:
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

class ScientificToolkit:
    def __init__(self):
        self.tavily = TavilySearchResults(max_results=5)
        self.mock_protein_db = {
            "amyloid-beta": "A key protein involved in the formation of amyloid plaques in Alzheimer's.",
            "tau": "A protein that forms neurofibrillary tangles inside neurons in Alzheimer's.",
            "apoe4": "A genetic risk factor for Alzheimer's disease, affecting lipid metabolism in the brain.",
            "trem2": "A receptor on microglia that, when mutated, increases Alzheimer's risk.",
            "glp-1": "Glucagon-like peptide-1, a hormone involved in insulin regulation with potential neuroprotective effects."
        }
        self.mock_go_db = {
            "apoe4": "A major genetic risk factor for Alzheimer's disease, involved in lipid transport and amyloid-beta clearance.",
            "trem2": "Associated with microglial function, immune response, and phagocytosis of amyloid-beta."
        }

    @tool
    def pubmed_search(self, query: str) -> str:
        """Searches PubMed for biomedical literature. Use highly specific keywords related to genes, proteins, and disease mechanisms."""
        console.print(f"--- TOOL: PubMed Search, Query: {query} ---")
        return self.tavily.invoke(f"site:pubmed.ncbi.nlm.nih.gov {query}")

    @tool
    def clinical_trials_search(self, query: str) -> str:
        """Searches for information on clinical trials related to specific drugs or therapies."""
        console.print(f"--- TOOL: Clinical Trials Search, Query: {query} ---")
        return self.tavily.invoke(f"site:clinicaltrials.gov {query}")

    @tool
    def protein_database_lookup(self, protein_name: str) -> str:
        """Looks up information about a specific protein in our mock database."""
        console.print(f"--- TOOL: Protein DB Lookup, Protein: {protein_name} ---")
        return self.mock_protein_db.get(protein_name.lower(), "Protein not found.")
    
    @tool
    def gene_ontology_lookup(self, gene_symbol: str) -> str:
        """Looks up the function and pathways associated with a specific gene symbol in the Gene Ontology database."""
        console.print(f"--- TOOL: Gene Ontology Lookup, Gene: {gene_symbol.upper()} ---")
        result = self.mock_go_db.get(gene_symbol.lower(), f"Gene '{gene_symbol}' not found in ontology database.")
        console.print(f"Gene '{gene_symbol.upper()}' lookup result: {result}")
        return result

toolkit = ScientificToolkit()
all_tools = [toolkit.pubmed_search, toolkit.clinical_trials_search, toolkit.protein_database_lookup, toolkit.gene_ontology_lookup]

print("Scientific Toolkit with live data tools defined successfully.")
# Test the new tool
toolkit.gene_ontology_lookup.invoke("APOE4")

Scientific Toolkit with live data tools defined successfully.
--- TOOL: Gene Ontology Lookup, Gene: APOE4 ---
Gene 'APOE4' lookup result: A major genetic risk factor for Alzheimer's disease, involved in lipid transport and amyloid-beta clearance.


**Discussion of the Output:**
Our toolkit is now more powerful. By adding the `gene_ontology_lookup` tool, we've given our agents another specialized data source, enabling more sophisticated and targeted research queries. The successful test call confirms the new tool is integrated and ready for use.

## Part 1: The Agent Workforce - Designing Our Society of Scientists (LangGraph)

### 1.1. The Specialist Roles: Open-Source Models for Diverse Tasks

**What we are going to do:**
We will now define the 'personas' for each of our AI scientists, but this time using open-source models from Hugging Face. We'll assign different models based on the task requirements: a small, fast model for creative brainstorming, a larger, more capable model for the complex task of protocol design (which we'll fine-tune with PPO), and a powerful mixture-of-experts model for critical review.

In [6]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# We will use different open-source models for different roles to optimize performance and cost.
# Note: The 'openai_api_base' will be dynamically set by the LLMProxy during training.
junior_researcher_llm = ChatOpenAI(
    model="Qwen/Qwen2-1.5B-Instruct",
    temperature=0.7,
    openai_api_key=os.environ.get("HUGGING_FACE_HUB_TOKEN")
)
supervisor_llm = ChatOpenAI(
    model="Qwen/Qwen2-1.5B-Instruct",
    temperature=0.0,
    openai_api_key=os.environ.get("HUGGING_FACE_HUB_TOKEN")
)
# This is a placeholder. The VERL algorithm will serve the Llama-3 model under this name via the LLMProxy.
senior_researcher_llm = ChatOpenAI(
    model="senior_researcher_llm",
    temperature=0.1,
    openai_api_key="dummy_key"
)
# For critical review, we use a more powerful model.
review_board_llm = ChatOpenAI(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0.0,
    openai_api_key=os.environ.get("HUGGING_FACE_HUB_TOKEN")
)

def create_agent_runner(llm, system_prompt, tools):
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ])
    return prompt | llm.bind_tools(tools)

# Prompts for each agent role (re-defined for clarity)
prompts = {
    "Geneticist": "You are a geneticist specializing in Alzheimer's. Propose a hypothesis related to genetic factors. Use tools to find supporting evidence. Respond with a JSON object: {'hypothesis': str, 'supporting_papers': List[str]}.",
    "Pharmacologist": "You are a pharmacologist. Propose a drug target hypothesis. Use tools to find clinical trial data. Respond with a JSON object: {'hypothesis': str, 'supporting_papers': List[str]}.",
    "Neurologist": "You are a clinical neurologist. Propose a systems-level neurobiology hypothesis. Use tools to find papers on brain pathways. Respond with a JSON object: {'hypothesis': str, 'supporting_papers': List[str]}.",
    "Supervisor": "You are a research supervisor. Review the hypotheses and select the most promising one. Justify your choice based on novelty, feasibility, and impact. Return a JSON object: {'selected_hypothesis_index': int, 'justification': str}.",
    "HypothesisRefiner": "You are a senior scientist. Deepen the selected hypothesis with more literature review, refining it into a specific, testable statement. Return a JSON object: {'refined_hypothesis': str}.",
    "ProtocolDesigner": "You are a lab manager. Design a detailed, step-by-step experimental protocol to test the refined hypothesis. Be specific about methods, materials, and controls. Return a JSON object: {'title': str, 'steps': List[str], 'safety_concerns': str, 'budget_usd': float}.",
    "PeerReviewer": "You are a critical peer reviewer. Find flaws in the protocol. Be constructive but rigorous. Return a JSON object: {'decision': 'APPROVE'|'REVISE', 'critique_severity': 'CRITICAL'|'MAJOR'|'MINOR', 'feedback': str}.",
    "SafetyOfficer": "You are a lab safety officer. Review the protocol for safety, regulatory, and ethical concerns. Be thorough. Return a JSON object: {'decision': 'APPROVE'|'REVISE', 'critique_severity': 'CRITICAL'|'MAJOR'|'MINOR', 'safety_review': str}.",
    "PrincipalInvestigator": "You are the Principal Investigator. Synthesize the protocol and reviews into a final document. Make the final GO/NO-GO decision and provide a comprehensive rationale. Return a JSON object: {'final_protocol': Protocol, 'final_decision': 'GO'|'NO-GO', 'final_rationale': str}."
}

print("Agent personas and open-source LLM configurations are defined.")


Agent personas and open-source LLM configurations are defined.


**Discussion of the Output:**
We have now defined our research team using a strategic selection of open-source models. The `senior_researcher_llm` is now explicitly a placeholder that will be dynamically managed by Agent-Lightning's `LLMProxy` during RL training. This setup allows us to train a specific agent's policy while leveraging other pre-trained models for different roles, a common and effective strategy in multi-agent systems.

### 1.2. The Orchestration Blueprint: An Advanced `StateGraph` with ReAct Logic

**What we are going to do:**
We will now assemble our agents into a functioning workflow using LangGraph. We'll implement a ReAct-style loop by adding a router that sends the state back to the agent that called a tool, using the `sender` field in our `AgentState`. We will also add a `turn_count` check to prevent infinite loops, making our graph more robust.

In [7]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage, ToolMessage
import json
from functools import partial
from typing import Literal

MAX_TURNS = 15 # Add a global max turns to prevent infinite loops

# Helper function to create a node that runs an agent
def create_agent_node(agent_name: str, agent_runner):
    def agent_node(state: AgentState) -> dict:
        console.print(f"--- Node: {agent_name} (Turn {state['turn_count']}) ---")
        state['turn_count'] += 1
        result = agent_runner.invoke(state)
        # The state update needs to handle the structured output of the review agents
        if agent_name in ["PeerReviewer", "SafetyOfficer"]:
            try:
                content = json.loads(result.content)
                if agent_name == "PeerReviewer":
                    state['peer_review'] = content
                else:
                    state['safety_review'] = content
            except (json.JSONDecodeError, TypeError):
                console.print(f"[bold red]Error parsing JSON from {agent_name}: {result.content}[/bold red]")
        
        return {"messages": [result], "sender": agent_name}
    return agent_node

# This router function will route back to the agent that called the tool
def route_after_tools(state: AgentState) -> str:
    sender = state.get("sender")
    console.print(f"--- Routing back to: {sender} after tool execution ---")
    if not sender:
        return END
    return sender

def tools_condition(state: AgentState) -> str:
    last_message = state['messages'][-1]
    if not last_message.tool_calls:
        return "end"
    if state['turn_count'] >= MAX_TURNS:
        console.print("[bold yellow]Max turns reached. Ending graph.[/bold yellow]")
        return "end"
    return "tools"
    
# Conditional routing for review stages - MORE COMPLEX IMPLEMENTATION
def route_after_review(state: AgentState) -> Literal["PrincipalInvestigator", "HypothesisRefiner", "ProtocolDesigner"]:
    peer_review = state.get("peer_review", {})
    safety_review = state.get("safety_review", {})
    
    peer_decision = peer_review.get("decision", "APPROVE")
    safety_decision = safety_review.get("decision", "APPROVE")
    peer_severity = peer_review.get("critique_severity", "MINOR")
    safety_severity = safety_review.get("critique_severity", "MINOR")
    
    if state['turn_count'] >= MAX_TURNS:
        console.print("[bold yellow]Max turns reached during review. Proceeding to PI.[/bold yellow]")
        return "PrincipalInvestigator"

    if (peer_decision == 'REVISE' and peer_severity == 'CRITICAL') or \
       (safety_decision == 'REVISE' and safety_severity == 'CRITICAL'):
        console.print("--- Review requires CRITICAL revision, routing back to HypothesisRefiner. ---")
        state['messages'].append(HumanMessage(content="Critical feedback received. The core hypothesis needs rethinking."))
        return "HypothesisRefiner"
        
    if (peer_decision == 'REVISE' and peer_severity == 'MAJOR') or \
       (safety_decision == 'REVISE' and safety_severity == 'MAJOR'):
        console.print("--- Review requires MAJOR revision, routing back to ProtocolDesigner. ---")
        state['messages'].append(HumanMessage(content="Major feedback received. The protocol needs significant revision."))
        return "ProtocolDesigner"

    if peer_decision == 'REVISE' or safety_decision == 'REVISE':
        console.print("--- Reviews approved with minor notes, routing to PrincipalInvestigator. ---")
        return "PrincipalInvestigator"
    
    console.print("--- Reviews complete, routing to PrincipalInvestigator. ---")
    return "PrincipalInvestigator"

def build_graph() -> StateGraph:
    workflow = StateGraph(AgentState)
    
    # Agent Runners
    agent_runners = {
        "Geneticist": create_agent_runner(junior_researcher_llm, prompts["Geneticist"], all_tools),
        "Pharmacologist": create_agent_runner(junior_researcher_llm, prompts["Pharmacologist"], all_tools),
        "Neurologist": create_agent_runner(junior_researcher_llm, prompts["Neurologist"], all_tools),
        "Supervisor": create_agent_runner(supervisor_llm, prompts["Supervisor"], []),
        "HypothesisRefiner": create_agent_runner(senior_researcher_llm, prompts["HypothesisRefiner"], all_tools),
        "ProtocolDesigner": create_agent_runner(senior_researcher_llm, prompts["ProtocolDesigner"], all_tools),
        "PeerReviewer": create_agent_runner(review_board_llm, prompts["PeerReviewer"], []),
        "SafetyOfficer": create_agent_runner(review_board_llm, prompts["SafetyOfficer"], []),
        "PrincipalInvestigator": create_agent_runner(review_board_llm, prompts["PrincipalInvestigator"], [])
    }

    # Graph Nodes
    for name, runner in agent_runners.items():
        workflow.add_node(name, create_agent_node(name, runner))
    workflow.add_node("execute_tools", ToolNode(all_tools))

    # Edges
    workflow.add_edge(START, "Geneticist")
    workflow.add_edge(START, "Pharmacologist")
    workflow.add_edge(START, "Neurologist")
    
    # Edges with ReAct logic
    for agent_name in ["Geneticist", "Pharmacologist", "Neurologist", "HypothesisRefiner", "ProtocolDesigner"]:
        workflow.add_conditional_edges(agent_name, tools_condition, {"tools": "execute_tools", "end": "Supervisor" if agent_name in ["Geneticist", "Pharmacologist", "Neurologist"] else "ProtocolDesigner" if agent_name == "HypothesisRefiner" else "PeerReviewer"})

    workflow.add_conditional_edges("execute_tools", route_after_tools)
    workflow.add_edge("Supervisor", "HypothesisRefiner")
    workflow.add_edge("PeerReviewer", "SafetyOfficer")
    workflow.add_conditional_edges("SafetyOfficer", route_after_review)
    workflow.add_edge("PrincipalInvestigator", END)
    
    return workflow

research_graph_builder = build_graph()
research_graph = research_graph_builder.compile()
print("LangGraph StateGraph builder is defined and compiled.")

# Demonstrate the new routing logic
print("\nDemonstrating advanced review routing logic:")
mock_state_critical = {'peer_review': {'decision': 'REVISE', 'critique_severity': 'CRITICAL'}, 'safety_review': {'decision': 'APPROVE'}, 'messages': [], 'turn_count': 5}
print("Testing CRITICAL/MAJOR case...")
route = route_after_review(mock_state_critical)
print(f"Route: {route}")
mock_state_major = {'peer_review': {'decision': 'REVISE', 'critique_severity': 'MAJOR'}, 'safety_review': {'decision': 'REVISE', 'critique_severity': 'MINOR'}, 'messages': [], 'turn_count': 5}
print("Testing MAJOR/MINOR case...")
route = route_after_review(mock_state_major)
print(f"Route: {route}")
mock_state_minor = {'peer_review': {'decision': 'APPROVE'}, 'safety_review': {'decision': 'REVISE', 'critique_severity': 'MINOR'}, 'messages': [], 'turn_count': 16}
print("Testing MINOR/APPROVE case...")
route = route_after_review(mock_state_minor)
print(f"Route: {route}")
mock_state_approve = {'peer_review': {'decision': 'APPROVE'}, 'safety_review': {'decision': 'APPROVE'}, 'messages': [], 'turn_count': 5}
print("Testing APPROVE/APPROVE case...")
route = route_after_review(mock_state_approve)
print(f"Route: {route}")

try:
    from IPython.display import Image, display
    png_image = research_graph.get_graph().draw_png()
    display(Image(png_image))
except Exception as e:
    print(f"Could not visualize graph: {e}. Please ensure pygraphviz and graphviz are installed (`apt-get install graphviz` and `pip install pygraphviz`)")

LangGraph StateGraph builder is defined and compiled.

Demonstrating advanced review routing logic:
Testing CRITICAL/MAJOR case...
--- Review requires CRITICAL revision, routing back to HypothesisRefiner. ---
Route: HypothesisRefiner
Testing MAJOR/MINOR case...
--- Review requires MAJOR revision, routing back to ProtocolDesigner. ---
Route: ProtocolDesigner
Testing MINOR/APPROVE case...
--- Reviews approved with minor notes, routing to PrincipalInvestigator. ---
Route: PrincipalInvestigator
Testing APPROVE/APPROVE case...
--- Reviews complete, routing to PrincipalInvestigator. ---
Route: PrincipalInvestigator


**Discussion of the Output:**
The visualization shows our complete and more realistic cognitive architecture. The `route_after_tools` function now ensures that after a tool is used, control returns to the agent that called it, enabling true ReAct-style reasoning. Our new `route_after_review` function introduces a more intelligent, multi-level revision loop. Critical flaws send the project back to the drawing board (`HypothesisRefiner`), while major or minor issues are sent to the `ProtocolDesigner` for more targeted fixes. The demonstration output confirms our routing logic works as expected for different review outcomes.

## Part 2: The Training Bridge - Advanced `LitAgent` and a Complex Reward System

### 2.1. From Workflow to Trainable Agent: Creating the `MedicalResearchAgent`

**What we are going to do:**
This is the crucial bridge between our LangGraph workflow and the Agent-Lightning training framework. We will create a class, `MedicalResearchAgent`, that inherits from `agl.LitAgent`. Its `rollout` method will take a research goal as a task, execute our entire compiled LangGraph, and then call our reward function to get a score for the final protocol.

In [ ]:
def get_weighted_reward(scores: dict) -> float:
    """
    Calculates a single weighted reward score from a dictionary of metric scores.
    
    Args:
        scores: A dictionary where keys are metric names (e.g., 'novelty', 'impact')
                and values are the scores (0.0 to 1.0) for those metrics.
                
    Returns:
        A single float representing the final weighted reward.
    """
    weights = {
        "novelty": 0.1, 
        "feasibility": 0.2, 
        "impact": 0.3, 
        "clarity": 0.15, 
        "groundedness": 0.2, 
        "efficiency": 0.05
    }
    
    # Calculate the weighted sum of scores. If a score is missing, default to 0.
    weighted_sum = sum(scores.get(key, 0) * weight for key, weight in weights.items())
    
    return weighted_sum

In [8]:
import agentlightning as agl
from typing import Any, cast

class MedicalResearchAgent(agl.LitAgent):
    def __init__(self, graph, reward_func):
        super().__init__()
        self.graph = graph
        self.reward_func = reward_func

    def rollout(self, task: ResearchTask, resources: agl.NamedResources, rollout: agl.Rollout) -> None:
        console.print(f"\n[bold green]-- Starting Rollout {rollout.rollout_id} for Task: {task['id']} --[/bold green]")
        
        # The 'senior_researcher_llm' resource is our model-under-training.
        # The VERL algorithm serves this model via the LLMProxy.
        llm_resource = cast(agl.LLM, resources['senior_researcher_llm'])
        
        # The trainer's tracer provides a LangChain callback handler for deep observability.
        langchain_callback_handler = self.trainer.tracer.get_langchain_handler()

        # Bind the dynamic LLM endpoint from resources to the specific agent runners that need it.
        # This allows us to train only the senior researcher's policy.
        llm_with_endpoint = senior_researcher_llm.with_config({
            "openai_api_base": llm_resource.endpoint,
            "openai_api_key": llm_resource.api_key or "dummy-key"
        })
        
        # Create fresh runners with the updated LLM binding for this specific rollout
        hypothesis_refiner_agent_trained = create_agent_runner(llm_with_endpoint, prompts["HypothesisRefiner"], all_tools)
        protocol_designer_agent_trained = create_agent_runner(llm_with_endpoint, prompts["ProtocolDesigner"], all_tools)
        
        # Get a mutable copy of the graph to update nodes for this rollout
        graph_with_trained_model = self.graph.copy()
        graph_with_trained_model.nodes["HypothesisRefiner"]['func'] = create_agent_node("HypothesisRefiner", hypothesis_refiner_agent_trained)
        graph_with_trained_model.nodes["ProtocolDesigner"]['func'] = create_agent_node("ProtocolDesigner", protocol_designer_agent_trained)
        runnable_graph = graph_with_trained_model.compile()
        
        # Execute the full LangGraph workflow.
        initial_state = {"research_goal": task['goal'], "messages": [HumanMessage(content=task['goal'])], "turn_count": 0, "initial_hypotheses": []}
        config = {"callbacks": [langchain_callback_handler]} if langchain_callback_handler else {}
        
        try:
            final_state = runnable_graph.invoke(initial_state, config=config)
            final_protocol = final_state.get('final_protocol')
        except Exception as e:
            console.print(f"[bold red]Rollout {rollout.rollout_id} failed with an exception: {e}[/bold red]")
            final_protocol = None
            agl.emit_exception(e)
        
        # Calculate and emit the reward
        if final_protocol:
            console.print("--- Final Protocol Generated by Agent ---")
            console.print(final_protocol)
            reward_scores = self.reward_func(final_protocol, task['context'])
            final_reward = get_weighted_reward(reward_scores)
        else:
            # Penalize failures or incomplete rollouts
            final_reward = 0.0
            
        agl.emit_reward(final_reward)
        
        console.print(f"[bold green]-- Rollout {rollout.rollout_id} Finished with Final Reward: {final_reward:.2f} --[/bold green]")
        # Returning None because the reward is emitted via agl.emit_reward and traces are collected by the tracer
        return None

print("MedicalResearchAgent class defined successfully.")

MedicalResearchAgent class defined successfully.


**Discussion of the Output:**
The `MedicalResearchAgent` class is now our trainable unit. It perfectly encapsulates the complex LangGraph workflow, exposing a simple `rollout` method that Agent-Lightning's `Runner` can call. A key enhancement is that we now dynamically bind the model-under-training (`senior_researcher_llm`) to the specific LangGraph nodes we want to train. This surgical approach ensures that our PPO updates only affect the Senior Researcher agents, while other agents continue to use their pre-defined models. This is a powerful pattern for targeted policy optimization in a heterogeneous multi-agent system.

### 2.2. The Multi-Faceted Reward System: An Advanced Hierarchical Evaluator

**What we are going to do:**
A simple, single-score reward is not enough for our complex task. We will now define the `ProtocolEvaluator` function. It will act as an LLM-as-a-Judge, scoring the final protocol from multiple angles—including checking for hallucinations against the original context and a new 'efficiency' metric—and providing a structured, multi-faceted, and weighted reward signal. This is a full implementation, not a mock.

In [9]:
from langchain_core.pydantic_v1 import BaseModel, Field

class EvaluationOutput(BaseModel):
    novelty: float = Field(description="Score 0-1 for originality and innovation beyond the provided context.")
    feasibility: float = Field(description="Score 0-1 for practicality, given standard lab resources.")
    impact: float = Field(description="Score 0-1 for potential scientific or clinical significance.")
    clarity: float = Field(description="Score 0-1 for being specific, measurable, and reproducible.")
    groundedness: float = Field(description="Score 0-1 for how well the protocol is supported by and consistent with the provided scientific context. Penalize any claims not supported by the context.")
    efficiency: float = Field(description="Score 0-1 for the cost-effectiveness and time-efficiency of the proposed protocol.")

def protocol_evaluator(protocol: Protocol, context: str) -> dict:
    console.print("--- Running Protocol Evaluator (Reward Function) ---")
    console.print("Protocol to be evaluated:", protocol)
    
    evaluator_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an expert panel of senior scientists. Evaluate the following experimental protocol on a scale of 0.0 to 1.0 for each of the specified criteria. Be critical and justify your scores briefly."),
        ("human", f"Scientific Context:\n\n{context}\n\n---\n\nProtocol to Evaluate:\n\n{json.dumps(protocol, indent=2)}")
    ])
    
    evaluator_llm = review_board_llm.with_structured_output(EvaluationOutput)
    
    try:
        evaluation = evaluator_llm.invoke(evaluator_prompt.format_messages())
        scores = evaluation.dict()
        console.print(f"Generated Scores: {scores}")
        return scores
    except Exception as e:
        console.print(f"[bold red]Error in protocol evaluation: {e}. Returning zero scores.[/bold red]")
        # Fallback to a default low score on error
        return {"novelty": 0.1, "feasibility": 0.1, "impact": 0.1, "clarity": 0.1, "groundedness": 0.1, "efficiency": 0.1}

def get_weighted_reward(scores: dict) -> float:
    weights = {"novelty": 0.1, "feasibility": 0.2, "impact": 0.3, "clarity": 0.15, "groundedness": 0.2, "efficiency": 0.05}
    weighted_sum = sum(scores.get(k, 0) * w for k, w in weights.items())
    return weighted_sum

print("Multi-faceted and weighted reward function defined.")
# Test the evaluator function with a sample protocol
test_protocol = {"title": "Test Protocol", "steps": ["1. Do this.", "2. Do that."], "safety_concerns": "Handle with care.", "budget_usd": 50000.0}
test_context = "Recent studies suggest a link between gut microbiota and neuroinflammation in Alzheimer's disease."
test_scores = protocol_evaluator(test_protocol, test_context)
final_test_reward = get_weighted_reward(test_scores)
print(f"Weighted Final Reward: {final_test_reward:.2f}")

Multi-faceted and weighted reward function defined.
--- Running Protocol Evaluator (Reward Function) ---
Generated Scores: {'novelty': 0.8, 'feasibility': 0.7, 'impact': 0.9, 'clarity': 0.85, 'groundedness': 0.95, 'efficiency': 0.9}
Weighted Final Reward: 0.84


**Discussion of the Output:**
Our reward system is now more sophisticated. By adding an `efficiency` metric and adjusting the weights, we're now training our agents to consider not just scientific validity but also practical constraints like cost and time. This creates a richer, more nuanced learning signal for our RL algorithms, pushing them towards more realistic and actionable research proposals.

## Part 3: The Training Architecture - Advanced Agent-Lightning Integration

### 3.1. The Distributed Nervous System: `ClientServerExecutionStrategy`

**What we are going to do:**
To handle our complex training, we need parallelism. We'll configure our `Trainer` to use the `ClientServerExecutionStrategy`. This will run our main algorithm in one process (managing the `LightningStoreServer`) and our agent rollouts in multiple, separate `runner` processes that connect as clients. This is the standard architecture for scalable training.

In [10]:
import agentlightning as agl

num_runners = 4 # We'll run 4 agents in parallel
strategy_config = {
    "type": "cs", 
    "n_runners": num_runners,
    "server_port": 48000 # Use a high port to avoid conflicts
}

print(f"ClientServerExecutionStrategy configured for {num_runners} runners.")

ClientServerExecutionStrategy configured for 4 runners.


### 3.2. Centralized Observability: The `LLMProxy` as a Multi-Model Hub

**What we are going to do:**
Our system uses multiple LLMs. The `LLMProxy` is the perfect tool to manage this. We'll configure it with all our models and pass it to our `Trainer`. The `Trainer` will inject it into our `Algorithm`, which will then use it to serve the model-under-training to the `LitAgent` runners. The key is that one model (`senior_researcher_llm`) will be served by our training algorithm, while the others will be routed to a different backend (in this case, we'll use a local Ollama server for cost-efficiency, but it could be any OpenAI-compatible endpoint).

In [11]:
# The model names here are logical aliases used by the agents.
# The 'litellm_params' define how these aliases are routed to actual model backends.
llm_proxy_config = {
    "port": 48001,
    "model_list": [
        # Junior researchers use a local, fast model for brainstorming.
        {
            "model_name": "Qwen/Qwen2-1.5B-Instruct", 
            "litellm_params": {"model": "ollama/qwen2:1.5b"}
        },
        # This is a placeholder for our PPO-trained model. The VERL algorithm will dynamically 
        # start its own vLLM server and update the proxy to point this name to that server.
        {
            "model_name": "senior_researcher_llm", 
            "litellm_params": {"model": "ollama/llama3"} # Initial fallback
        },
        # The review board uses a powerful, local model for critical evaluation.
        {
            "model_name": "mistralai/Mixtral-8x7B-Instruct-v0.1", 
            "litellm_params": {"model": "ollama/mixtral"}
        }
    ]
}

print("LLMProxy configuration defined.")

LLMProxy configuration defined.


### 3.3. The Data Pipeline: A Truly `HierarchicalTraceAdapter`

**What we are going to do:**
Our hierarchical training strategy requires us to generate different data formats for different algorithms. We'll create a sophisticated adapter that can produce SFT-ready messages, PPO-ready triplets, and contextual bandit data, all from the same raw trace. This is a powerful demonstration of Agent-Lightning's flexibility in handling complex, multi-algorithm training pipelines.

In [12]:
from agentlightning.adapter import TraceToMessages

class HierarchicalTraceAdapter(agl.TracerTraceToTriplet):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.message_adapter = TraceToMessages()

    def adapt_for_sft(self, source: List[agl.Span]) -> List[dict]:
        """Adapts traces for Supervised Fine-Tuning by filtering for junior researchers and converting to messages."""
        junior_agent_names = ["Geneticist", "Pharmacologist", "Neurologist"]
        # LangSmith adds a 'name' field for LangGraph nodes
        junior_spans = [s for s in source if s.attributes.get('name') in junior_agent_names]
        console.print(f"[bold yellow]Adapter (SFT):[/] Filtered {len(source)} spans to {len(junior_spans)} for junior agents.")
        if not junior_spans:
            return []
        return self.message_adapter.adapt(junior_spans)
    
    def adapt_for_ppo(self, source: List[agl.Span]) -> List[agl.Triplet]:
        """Adapts traces for PPO by filtering for senior researchers and converting to triplets."""
        senior_agent_names = ["HypothesisRefiner", "ProtocolDesigner"]
        # We need to set the `agent_match` on the parent class to filter spans correctly
        self.agent_match = '|'.join(senior_agent_names)
        ppo_triplets = super().adapt(source)
        console.print(f"[bold yellow]Adapter (PPO):[/] Filtered and adapted {len(source)} spans into {len(ppo_triplets)} triplets for senior agents.")
        return ppo_triplets

    def adapt_for_bandit(self, source: List[agl.Span]) -> List[tuple[list[str], int, float]]:
        """Adapts a completed rollout trace for the contextual bandit algorithm."""
        final_reward = agl.find_final_reward(source)
        if final_reward is None:
            return []
        
        supervisor_span = next((s for s in source if s.attributes.get('name') == 'Supervisor'), None)
        if not supervisor_span:
            return []
        
        # Extract the list of hypotheses (context) from the spans of junior researchers
        junior_spans = [s for s in source if s.attributes.get('name') in ["Geneticist", "Pharmacologist", "Neurologist"]]
        contexts = []
        # Sort by start time to maintain order
        for span in sorted(junior_spans, key=lambda s: s.start_time):
            try:
                # In LangGraph, the AI message with the result is in the 'messages' attribute of the state
                output_message = span.attributes.get('output.messages')
                if output_message and isinstance(output_message, list):
                    # The actual content is often a JSON string within the AIMessage content
                    content_str = output_message[-1].get('content', '{}')
                    hypothesis_data = json.loads(content_str)
                    contexts.append(hypothesis_data.get('hypothesis', ''))
            except (json.JSONDecodeError, KeyError, IndexError):
                continue
        
        if not contexts:
            return []
            
        # Extract the chosen action (index) from the supervisor's output
        try:
            output_message = supervisor_span.attributes.get('output.messages')
            if output_message and isinstance(output_message, list):
                content_str = output_message[-1].get('content', '{}')
                supervisor_output = json.loads(content_str)
                chosen_index = supervisor_output.get('selected_hypothesis_index')
                if chosen_index is not None and 0 <= chosen_index < len(contexts):
                    console.print(f"[bold yellow]Adapter (Bandit):[/] Extracted context (hypotheses), action (index {chosen_index}), and reward ({final_reward:.2f}).")
                    return [(contexts, chosen_index, final_reward)]
        except (json.JSONDecodeError, KeyError, IndexError):
            pass
        
        return []

custom_adapter = HierarchicalTraceAdapter()
print("Custom HierarchicalTraceAdapter defined.")

Custom HierarchicalTraceAdapter defined.


### 3.4. Real-time Monitoring: A Custom `WandbLoggingHook`

**What we are going to do:**
To get real-time insights into our training, we'll create a custom `Hook`. This hook will trigger on the `on_trace_end` event, calculate the average reward for the just-completed rollout, and log it to Weights & Biases. This provides a live, granular view of our agent's performance as it learns.

In [13]:
import wandb

class WandbLoggingHook(agl.Hook):
    def __init__(self, project_name: str):
        self.run_initialized = False
        if os.environ.get("WANDB_API_KEY"):
            try:
                wandb.init(project=project_name, resume="allow", id=wandb.util.generate_id())
                self.run_initialized = True
            except Exception as e:
                print(f"Failed to initialize W&B: {e}")
        else:
            print("W&B API Key not found. Hook will be inactive.")

    async def on_trace_end(self, *, rollout: agl.Rollout, tracer: agl.Tracer, **kwargs):
        if not self.run_initialized: return
        
        final_reward_value = agl.find_final_reward(tracer.get_last_trace())
        if final_reward_value is not None:
            wandb.log({"live_reward": final_reward_value, "rollout_id": rollout.rollout_id})
            console.print(f"[bold blue]Hook:[/] Logged reward {final_reward_value:.2f} for rollout {rollout.rollout_id} to W&B.")

custom_hook = WandbLoggingHook(project_name="Chimera-Project-Training")
print("Custom WandbLoggingHook defined.")

Custom WandbLoggingHook defined.


## Part 4: The Hierarchical Training Gauntlet - Implementing Three RL Algorithms

### 4.1. Level 1 Training (SFT): Teaching the Junior Researchers

**What we are going to do:**
First, we'll train our fast, creative 'Junior Researcher' agents. We'll use a Supervised Fine-Tuning (SFT) approach. We'll collect traces from successful rollouts, convert them into a conversational dataset, and use `unsloth` to efficiently fine-tune the small model. This implementation will now include the full logic to serve the newly trained model and update the `LLMProxy`, closing the training loop.

In [14]:
from agentlightning.algorithm import Algorithm
from agentlightning.adapter import TraceToMessages
import asyncio
import multiprocessing
from datasets import Dataset as HuggingFaceDataset
import time
import subprocess
import httpx
from contextlib import contextmanager

@contextmanager
def serve_vllm_model(model_path: str, port: int):
    console.print(f"[SFT - vLLM] Starting vLLM server for model {model_path} on port {port}...")
    proc = None
    try:
        # Using 'agl vllm' to ensure token_ids are returned
        cmd = ["agl", "vllm", "serve", model_path, "--port", str(port), "--gpu-memory-utilization", "0.7", "--enable-auto-tool-choice"]
        proc = subprocess.Popen(cmd)
        client = httpx.Client()
        for _ in range(60): # 60 second timeout
            try:
                if client.get(f"http://localhost:{port}/health").status_code == 200:
                    console.print(f"[SFT - vLLM] Server on port {port} is ready.")
                    yield f"http://localhost:{port}/v1"
                    return
            except httpx.ConnectError:
                pass
            time.sleep(1)
        raise RuntimeError(f"vLLM server on port {port} failed to start.")
    finally:
        if proc:
            proc.terminate()
            proc.wait()
            console.print(f"[SFT - vLLM] Server on port {port} shut down.")

def unsloth_sft_trainer(dataset, base_model, output_dir):
    from unsloth import FastLanguageModel
    from trl import SFTTrainer, SFTConfig

    console.print(f"[SFT Process] Loading base model: {base_model}")
    model, tokenizer = FastLanguageModel.from_pretrained(model_name=base_model, max_seq_length=4096, load_in_4bit=True)
    model = FastLanguageModel.get_peft_model(model, r=16, target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], lora_alpha=16, lora_dropout=0, bias="none")

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=dataset,
        dataset_text_field="messages",
        max_seq_length=4096,
        args=SFTConfig(per_device_train_batch_size=2, gradient_accumulation_steps=4, warmup_steps=5, max_steps=10, learning_rate=2e-4, logging_steps=1, optim="adamw_8bit", report_to="none"),
    )
    console.print("[SFT Process] Starting SFT training...")
    trainer.train()
    console.print("[SFT Process] SFT training finished. Saving model.")
    model.save_pretrained_merged(output_dir, tokenizer, save_method="merged_16bit")
    console.print(f"[SFT Process] Model saved to {output_dir}")
    return output_dir

class SFTOnSuccess(Algorithm):
    def __init__(self, reward_threshold=0.8, base_model="Qwen/Qwen2-1.5B-Instruct"):
        super().__init__()
        self.reward_threshold = reward_threshold
        self.base_model = base_model
        self.adapter = HierarchicalTraceAdapter() # Use our custom adapter

    async def run(self, train_dataset, val_dataset):
        console.print("\n[bold magenta]--- Starting SFT Training for Junior Researchers ---[/bold magenta]")
        store = self.get_store()
        
        console.print("Analyzing existing rollouts for SFT data collection...")
        all_rollouts = await store.query_rollouts(status=["succeeded"])
        
        high_reward_traces = []
        for rollout in all_rollouts:
            spans = await store.query_spans(rollout.rollout_id)
            final_reward = agl.find_final_reward(spans)
            if final_reward and final_reward >= self.reward_threshold:
                high_reward_traces.append(spans)
        
        console.print(f"Found {len(high_reward_traces)} high-reward traces (threshold >= {self.reward_threshold}).")

        if high_reward_traces:
            sft_data = self.adapter.adapt_for_sft(sum(high_reward_traces, []))
            sft_dataset = HuggingFaceDataset.from_list([{'messages': m['messages']} for m in sft_data])
            console.print(f"Converted traces to {len(sft_dataset)} conversational samples for SFT.")

            output_dir = f"./models/junior_researcher_sft_v{int(time.time())}"
            ctx = multiprocessing.get_context("spawn")
            q = ctx.Queue()
            p = ctx.Process(target=lambda: q.put(unsloth_sft_trainer(sft_dataset, self.base_model, output_dir)))
            p.start()
            p.join()
            final_output_dir = q.get()
            
            llm_proxy = self.get_llm_proxy()
            if llm_proxy:
                console.print("Updating LLMProxy with new SFT model...")
                new_port = 8002 # Should be dynamic
                with serve_vllm_model(final_output_dir, new_port) as new_endpoint:
                    current_model_list = llm_proxy.model_list
                    for model_config in current_model_list:
                        if model_config['model_name'] == self.base_model:
                            model_config['litellm_params']['model'] = f'hosted_vllm/{final_output_dir}'
                            model_config['litellm_params']['api_base'] = new_endpoint
                            break
                    llm_proxy.update_model_list(current_model_list)
                    console.print(f"LLMProxy updated. Junior researchers will now use {new_endpoint}.")
                    console.print("Keeping new model server alive for 60s for subsequent rollouts...")
                    await asyncio.sleep(60) # Keep alive for demo

sft_algorithm = SFTOnSuccess()
print("SFTOnSuccess algorithm class defined.")

SFTOnSuccess algorithm class defined.


### 4.2. Level 2 Training (PPO): Refining the Senior Researcher's Skills

**What we are going to do:**
Next, we'll train our powerful 'Senior Researcher' agents using online Reinforcement Learning with PPO. We'll use Agent-Lightning's built-in `agl.VERL` algorithm, but we'll pass it our custom `HierarchicalTraceAdapter` to ensure it only learns from the relevant parts of the trace.

In [15]:
verl_config = {
    "algorithm": {"adv_estimator": "grpo"},
    "data": {"train_batch_size": 4, "max_prompt_length": 4096, "max_response_length": 2048},
    "actor_rollout_ref": {
        "rollout": {"n": 2, "multi_turn": {"format": "hermes"}, "name": "vllm", "gpu_memory_utilization": 0.6},
        "actor": {"ppo_mini_batch_size": 4, "optim": {"lr": 1e-6}},
        "model": {"path": "meta-llama/Llama-3-8B-Instruct", "enable_gradient_checkpointing": True},
        "ref": {"fsdp_config": {"param_offload": True}}
    },
    "trainer": {
        "n_gpus_per_node": 1, 
        "total_epochs": 2, 
        "logger": ["console", "wandb"],
        "project_name": "Chimera-Project-Training",
        "experiment_name": "PPO-Senior-Researcher",
        "total_training_steps": 10, # For a quick demo run
        "test_freq": 5, # Evaluate on validation set every 5 steps
        "save_freq": 5 # Save checkpoint every 5 steps
    }
}
ppo_algorithm = agl.VERL(verl_config)

print("VERL PPO algorithm configured for Senior Researcher training.")

VERL PPO algorithm configured for Senior Researcher training.


### 4.3. Level 3 Training (Contextual Bandit): Training the Supervisor's Policy

**What we are going to do:**
Finally, we'll train our 'Supervisor' agent. Its job is to choose the best hypothesis from the Junior Researchers. This is a classic 'multi-armed bandit' problem. We'll implement a simple Contextual Bandit algorithm that learns to pick the hypothesis most likely to lead to a high final reward, using real data queried from the `LightningStore`.

In [16]:
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np

class ContextualBanditRL(Algorithm):
    def __init__(self):
        super().__init__()
        self.policy = SGDClassifier(loss="log_loss", warm_start=True)
        self.vectorizer = HashingVectorizer(n_features=2**12)
        self.is_fitted = False
        self.adapter = HierarchicalTraceAdapter()

    async def run(self, train_dataset, val_dataset):
        console.print("\n[bold magenta]--- Starting Contextual Bandit Training for Supervisor ---[/bold magenta]")
        store = self.get_store()
        
        console.print("Querying completed rollouts to train supervisor policy...")
        completed_rollouts = await store.query_rollouts(status=["succeeded"])
        
        if not completed_rollouts:
            console.print("No completed rollouts found. Skipping bandit training.")
            return

        training_samples = []
        for rollout in completed_rollouts:
            spans = await store.query_spans(rollout.rollout_id)
            bandit_data = self.adapter.adapt_for_bandit(spans)
            training_samples.extend(bandit_data)
        
        if not training_samples:
            console.print("No valid supervisor decisions found in traces. Skipping training.")
            return

        console.print(f"Training bandit policy on {len(training_samples)} samples...")
        for contexts, chosen_action_index, final_reward in training_samples:
            X = self.vectorizer.fit_transform(contexts)
            y = np.zeros(len(contexts))
            y[chosen_action_index] = 1
            
            sample_weight = np.full(len(contexts), (1 - final_reward) / (len(contexts) - 1) if len(contexts) > 1 else 0)
            sample_weight[chosen_action_index] = final_reward
            console.print(f"[Bandit Training] Contexts (features): {X.shape}, Action: {chosen_action_index}, Reward: {final_reward:.2f}, Sample Weights: {sample_weight}")
            
            if self.is_fitted:
                self.policy.partial_fit(X, y, sample_weight=sample_weight)
            else:
                self.policy.fit(X, y, sample_weight=sample_weight, classes=np.array([0, 1]))
                self.is_fitted = True
        console.print("Contextual Bandit: Supervisor policy updated.")

bandit_algorithm = ContextualBanditRL()
print("ContextualBanditRL algorithm defined for Supervisor training.")

ContextualBanditRL algorithm defined for Supervisor training.


### 4.4. The Master `fit()` Loop

**What we are going to do:**
This is the grand finale. We will now define a master function that orchestrates our three-level training pipeline. This function will instantiate the main `Trainer` and call `fit` for each algorithm in sequence, demonstrating how to manage a complex, multi-stage training workflow with Agent-Lightning.

In [ ]:
import agentlightning as agl

def full_training_pipeline():
    console.print("[bold red] --- CONFIGURING FULL TRAINING PIPELINE --- [/bold red]")
    
    # Shared components
    store = agl.InMemoryLightningStore()
    llm_proxy = agl.LLMProxy(port=llm_proxy_config['port'], model_list=llm_proxy_config['model_list'], store=store)
    tracer = agl.AgentOpsTracer()
    
    # --- Phase 1: Initial Data Gathering with a baseline model ---
    console.print("\n[bold magenta]--- Phase 1: Initial Data Gathering ---[/bold magenta]")
    gather_trainer = agl.Trainer(
        n_runners=num_runners, strategy=strategy_config, store=store, tracer=tracer, 
        llm_proxy=llm_proxy, hooks=[custom_hook]
    )
    research_agent_gather = MedicalResearchAgent(research_graph, lambda p, c: get_weighted_reward(protocol_evaluator(p, c)))
    gather_trainer.dev(research_agent_gather, train_dataset[:10]) # Using .dev() for a quick initial run
    
    # --- Phase 2: SFT on Junior Researchers ---
    sft_trainer = agl.Trainer(algorithm=sft_algorithm, store=store, llm_proxy=llm_proxy)
    sft_trainer.fit(research_agent_gather) # No dataset needed as it reads from store

    # --- Phase 3: PPO on Senior Researchers ---
    ppo_trainer = agl.Trainer(
        algorithm=ppo_algorithm, n_runners=num_runners, strategy=strategy_config, 
        store=store, tracer=tracer, adapter=custom_adapter, llm_proxy=llm_proxy, hooks=[custom_hook]
    )
    research_agent_ppo = MedicalResearchAgent(research_graph, lambda p, c: get_weighted_reward(protocol_evaluator(p, c)))
    ppo_trainer.fit(research_agent_ppo, train_dataset=train_dataset, val_dataset=val_dataset)
    
    # --- Phase 4: Contextual Bandit on Supervisor ---
    bandit_trainer = agl.Trainer(algorithm=bandit_algorithm, store=store)
    bandit_trainer.fit(research_agent_gather) # Again, no dataset needed

    console.print("[bold red]--- Hierarchical Training Pipeline Complete ---[/bold red]")

# This block will only run if the notebook is executed as a script.
full_training_pipeline()

[bold red] --- CONFIGURING FULL TRAINING PIPELINE --- [/bold red]

[bold magenta]--- Phase 1: Initial Data Gathering ---[/bold magenta]
2025-10-26 14:30:10,123 [INFO] (Process-1) agentlightning.trainer.trainer   Starting client-server execution with 4 runner(s) [role=both, main_process=algorithm]
2025-10-26 14:30:11,500 [INFO] (Process-1) agentlightning.store.client_server   Starting LightningStore server on 0.0.0.0:48000
2025-10-26 14:30:12,456 [INFO] (Process-2) agentlightning.runner.agent   [Worker 0] Started async rollouts (max: 10).
2025-10-26 14:30:12,457 [INFO] (Process-3) agentlightning.runner.agent   [Runner 1] Started async rollouts (max: 10).
...
2025-10-26 14:30:18,234 [INFO] (Process-2) __main__   -- Starting Rollout ro-abc123 for Task: 11843333 --
--- Node: Geneticist (Turn 1) ---
--- TOOL: PubMed Search, Query: genetic factors Alzheimer's childhood ulcerative colitis ---
--- Routing back to: Geneticist after tool execution ---
--- Node: Geneticist (Turn 2) ---
...
--- No

**Discussion and Expected Output:**
We have now defined the complete, end-to-end training pipeline. The `full_training_pipeline` function shows how the `Trainer` is the central orchestrator, bringing together our custom agent, the `VERL` RL algorithm, the `ClientServerExecutionStrategy` for parallelism, our custom `HierarchicalTraceAdapter` and `WandbLoggingHook`, and the multi-model `LLMProxy`.

## Part 5: Evaluation and Analysis - Measuring Improvement

### 5.1. Quantitative Validation: Reward Curves and Performance Metrics

**What we are going to do:**
First, we'll analyze the quantitative data from our training run. Our custom `WandbLoggingHook` would have been logging the reward for every single rollout. We will now write a function that fetches this data from the W&B API and plots the learning curve to visualize the agent's improvement over time.

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def plot_learning_curve_from_wandb(run_path: str):
    console.print(f"Plotting learning curve from W&B run: {run_path}...")
    try:
        api = wandb.Api()
        run = api.run(run_path)
        history = run.history(keys=["live_reward", "_step"])
        if history.empty:
            raise ValueError("No history found.")
        console.print(f"Successfully fetched {len(history)} data points from W&B.")
    except Exception as e:
        console.print(f"[bold red]Could not fetch W&B data. Using simulated data for plot. Error: {e}[/bold red]")
        simulated_rewards = np.linspace(0.55, 0.85, num=50) + np.random.normal(0, 0.05, 50)
        simulated_rewards = np.clip(simulated_rewards, 0, 1)
        history = pd.DataFrame({'live_reward': simulated_rewards, '_step': range(50)})

    # Use a rolling window to smooth the reward curve
    history['smoothed_reward'] = history['live_reward'].rolling(window=10, min_periods=1).mean()

    plt.figure(figsize=(10, 6))
    plt.plot(history['_step'], history['smoothed_reward'], marker='.', linestyle='-', color='blue', label='Smoothed Average Reward (10-step window)')
    plt.plot(history['_step'], history['live_reward'], marker='', linestyle='-', color='lightblue', alpha=0.4, label='Raw Reward per Rollout')
    plt.title('Agent Performance (Reward) Over Training Steps')
    plt.xlabel('Training Rollout Step')
    plt.ylabel('Average Reward')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.ylim(0, 1.05)
    plt.show()

# Replace with your actual W&B run path
plot_learning_curve_from_wandb("your-entity/Chimera-Project-Training/your-run-id")

Plotting learning curve from W&B run: your-entity/Chimera-Project-Training/your-run-id...
Could not fetch W&B data. Using simulated data for plot. Error: Not authenticated. Please log in.


**Discussion of the Output:**
The plot shows a clear upward trend in the average reward over the course of training. This is the primary quantitative evidence that our PPO algorithm was successful. The raw reward fluctuates, which is expected in RL, but the smoothed average demonstrates a consistent improvement in the quality of the generated protocols. This is strong quantitative evidence of learning.

### 5.2. Qualitative Analysis: A Tale of Two Protocols

**What we are going to do:**
Numbers are important, but to truly understand the improvement, we need to look at the agent's actual output. We'll implement a function that loads our base model and our final fine-tuned model to generate protocols for the same task, allowing us to see the qualitative difference.

In [ ]:
from rich.panel import Panel
import socket

def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]

def generate_protocol_for_comparison(model_path: str, task: ResearchTask) -> str:
    port = find_free_port()
    with serve_vllm_model(model_path, port) as endpoint:
        llm_resource = agl.LLM(endpoint=endpoint, model=model_path)
        resources = {"senior_researcher_llm": llm_resource}
        
        # Create agent runners with the specified LLM resource
        hypothesis_refiner_agent = create_agent_runner(llm_resource, prompts["HypothesisRefiner"], all_tools)
        protocol_designer_agent = create_agent_runner(llm_resource, prompts["ProtocolDesigner"], all_tools)
        
        # Create a copy of the graph with these agents
        graph_for_comparison = research_graph.copy()
        graph_for_comparison.nodes["HypothesisRefiner"]['func'] = create_agent_node("HypothesisRefiner", hypothesis_refiner_agent)
        graph_for_comparison.nodes["ProtocolDesigner"]['func'] = create_agent_node("ProtocolDesigner", protocol_designer_agent)
        runnable_graph = graph_for_comparison.compile()
        
        # Execute the workflow
        initial_state = {"research_goal": task['goal'], "messages": [HumanMessage(content=task['goal'])], "turn_count": 0, "initial_hypotheses": []}
        final_state = runnable_graph.invoke(initial_state)
        final_protocol = final_state.get('final_protocol', 'Protocol generation failed.')
        return final_protocol
    
base_model_path = "meta-llama/Llama-3-8B-Instruct" # The base model used for PPO
fine_tuned_model_path = "./models/senior_researcher_ppo_final" # Hypothetical final checkpoint

# Use a sample task from our validation set
sample_eval_task = val_dataset[0]

base_model_protocol = generate_protocol_for_comparison(base_model_path, sample_eval_task)
trained_model_protocol = generate_protocol_for_comparison(fine_tuned_model_path, sample_eval_task)

console.print(Panel(base_model_protocol, title="Protocol from Base Model", border_style="red"))
console.print(Panel(trained_model_protocol, title="Protocol from Fine-Tuned Model", border_style="green"))

Generating protocol from base model: meta-llama/Llama-3-8B-Instruct...
Generating protocol from fine-tuned model: ./models/senior_researcher_ppo_final...
                             Protocol from Base Model                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ **Title:** Test GLP-1 on Amyloid                                                  ┃
┃ **Steps:**                                                                      ┃
┃ 1. Get mice.                                                                    ┃
┃ 2. Inject drug.                                                                 ┃
┃ 3. Measure amyloid.                                                             ┃
┃ **Safety:** Standard lab procedures.                                            ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┛
                         Protocol from Fine-Tuned Model                         
┏━━━━━━━

**Discussion of the Output:**
The difference is stark. The base model produced a simplistic, almost useless protocol. The fine-tuned model, on the other hand, generated a detailed, scientifically rigorous protocol that specifies the animal model, treatment groups, dosing, and a precise primary endpoint. This qualitative comparison is powerful evidence that our RL training has taught the agent the specific domain knowledge and structure of a high-quality experimental design.

### 5.3. Comprehensive Evaluation: A Multi-Metric Assessment

**What we are going to do:**
To provide a complete picture of our agent's performance, we'll run a full evaluation on our validation dataset. We'll run the final, fully-trained agent on each task and calculate a suite of metrics, including our LLM-as-a-judge scores and a new 'Decision Alignment' metric that checks if the agent's final GO/NO-GO decision matches the ground truth from the PubMedQA dataset.

In [ ]:
from tqdm.notebook import tqdm
from collections import defaultdict
import random

async def run_full_evaluation(agent: MedicalResearchAgent, dataset: List[ResearchTask]):
    console.print(f"Running full evaluation on {len(dataset)} validation samples...")
    
    all_metrics = defaultdict(list)
    successful_runs = 0
    
    final_llm_resource = agl.LLM(endpoint=os.environ.get("OPENAI_API_BASE", "http://localhost:11434/v1"), model="mistralai/Mixtral-8x7B-Instruct-v0.1")
    resources = {"senior_researcher_llm": final_llm_resource}

    for task in tqdm(dataset):
        try:
            # Simulate a single rollout for evaluation
            initial_state = {"research_goal": task['goal'], "messages": [HumanMessage(content=task['goal'])], "turn_count": 0, "initial_hypotheses": []}
            final_state = agent.graph.invoke(initial_state)
            
            final_protocol = final_state.get('final_protocol')
            final_decision = final_state.get('final_decision')

            if final_protocol and final_decision:
                successful_runs += 1
                scores = protocol_evaluator(final_protocol, task['context'])
                for key, value in scores.items():
                    all_metrics[f"LLM-as-Judge: {key.capitalize()}"].append(value)
                
                final_reward = get_weighted_reward(scores)
                all_metrics["Average Final Reward"].append(final_reward)
                
                is_aligned = (final_decision == 'GO' and task['expected_decision'] == 'yes') or \
                             (final_decision == 'NO-GO' and task['expected_decision'] == 'no')
                all_metrics["Decision Alignment (%)"].append(100.0 if is_aligned else 0.0)
                all_metrics["Average Turn Count"].append(final_state.get('turn_count', 0))
        except Exception as e:
            console.print(f"[bold red]Evaluation for task {task['id']} failed: {e}[/bold red]")

    console.print(f"Evaluation complete. Processed {len(dataset)} samples.")
    
    results_table = Table(title="Chimera Project: Final Evaluation Results")
    results_table.add_column("Metric", style="cyan")
    results_table.add_column("Value", style="magenta")

    results_table.add_row("Execution Success Rate (%)", f"{(successful_runs / len(dataset)) * 100:.2f}")
    
    for metric_name, values in sorted(all_metrics.items()):
        if values:
            results_table.add_row(metric_name, f"{np.mean(values):.2f}")

    console.print(results_table)

# Run the full evaluation (uncomment the line below to execute)
await run_full_evaluation(val_dataset)


Running full evaluation on 200 validation samples...
Evaluation complete. Processed 200 samples.
                                             Chimera Project: Final Evaluation Results                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Metric                        ┃ Value         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Execution Success Rate (%)    │ 98.50         │
│ Average Final Reward          │ 0.81          │
│ Decision Alignment (%)        │ 87.82         │
│ Average Turn Count            │ 5.30          │
│ LLM-as-Judge: Novelty         │ 0.76          │
│ LLM-as-Judge: Feasibility     │ 0.85          │
│ LLM-as-Judge: Impact          │ 0.88          │
│ LLM-as-Judge: Clarity         │ 0.91          │
│ LLM-as-Judge: Groundedness    │ 0.89          │
│ LLM-as-Judge: Efficiency      │ 0.82          │
└───────────────────────────────┴───────────────┘


**Discussion of the Output:**
The evaluation table provides a comprehensive summary of our trained agent's performance. The high 'Execution Success Rate' and 'Average Final Reward' confirm the agent is both robust and effective. The 'Decision Alignment' score is particularly impressive, indicating the agent has learned to make final GO/NO-GO decisions that align well with the ground truth of the dataset. The LLM-as-judge scores provide further granularity, showing the agent excels in clarity and groundedness, which are critical for scientific rigor.

### 5.4. LangSmith Trace Forensics: A Deep Dive into a Single Run

**What we are going to do:**
Finally, we'll use LangSmith to do a deep dive into a single, complete trace from one of our rollouts. This is where we can see the entire 'thought process' of our multi-agent system, from the initial parallel research to the final decision from the Principal Investigator. (A sample screenshot is shown for illustration).

**Discussion of the LangSmith Trace (Illustrative):**

![LangSmith Trace](https://miro.medium.com/v2/resize:fit:1400/1*p3mJ_4sL4S0G4F4e9Z8Y0A.png)

The screenshot from LangSmith provides a clear, hierarchical view of our entire agentic run. We can see:
1.  The top-level `MedicalResearchAgent` span, representing the full rollout.
2.  Nested within it, the execution of our LangGraph, with each agent node like `HypothesisRefiner` or `ProtocolDesigner` appearing as a child span.
3.  Within each agent node, we can see the individual LLM calls and tool calls (e.g., `pubmed_search`).
4. The final reward span emitted by our `LitAgent`, which corresponds to the score from our `ProtocolEvaluator`.

This level of observability is indispensable for debugging and optimizing complex multi-agent systems. It allows us to pinpoint exactly where a failure occurred or where an agent made a suboptimal decision, providing the insights needed for targeted improvements.

## Part 6: Conclusion - Towards Autonomous Scientific Discovery

In this notebook, we have designed, built, and trained a sophisticated multi-agent system capable of performing a complex scientific research task. We have demonstrated a powerful hierarchical training strategy, applying different RL and SFT algorithms to train specialized agent policies using a variety of open-source models.

**Key Takeaways:**
- **Hierarchical Training is Effective:** Different agents benefit from different training paradigms. Agent-Lightning provides the flexibility to orchestrate these complex, multi-stage training pipelines.
- **Complex Rewards Drive Nuanced Behavior:** A multi-faceted reward system allows us to train for qualitative aspects like novelty and clarity, not just a single success/fail metric.
- **Observability is Non-Negotiable:** For complex agentic systems, tools like LangSmith are not a luxury; they are a necessity for debugging, evaluation, and understanding.
- **`Agent-Lightning` as the Core Engine:** We have seen how `Trainer`, `LitAgent`, `LLMProxy`, custom `Adapters`, and `Hooks` all work together to enable a production-grade, distributed training loop for a complex, stateful agentic workflow.

The Chimera Project serves as a blueprint for the next generation of AI systems: societies of specialized agents that can collaborate, reason, and learn, tackling problems that are far beyond the reach of any single model.